<a href="https://colab.research.google.com/github/KartohaWhy/my_colab/blob/main/LLM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговое домашнее задание курса

В этом задании Вы потренируетесь применять различные модели из HuggingFace для решения задач NLP, а также поработаете с LLM и углубитесь в особенности промптинга для взаимодействия с моделями.

## Часть 1: тренируемся применять предобученные модели для решения различных NLP-задач

Загрузим датасет cnn-dailymail с ежедневными новостями.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/newspaper-text-summarization-cnn-dailymail


In [ ]:
import os
for file in os.listdir(path):
    print("📄", file)

📄 cnn_dailymail


In [ ]:
inner_path = os.path.join(path, "cnn_dailymail")
for file in os.listdir(inner_path):
    print("📄", file)

📄 validation.csv
📄 train.csv
📄 test.csv


In [ ]:
import pandas as pd

df = pd.read_csv(os.path.join(inner_path, "train.csv"))
print(df.columns)
print(df.head())

Index(['id', 'article', 'highlights'], dtype='object')
                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   
1  0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2  00027e965c8264c35cc1bc55556db388da82b07f   
3  0002c17436637c4fe1837c935c04de47adb18e9a   
4  0003ad6ef0c37534f80b55b4235108024b407f0b   

                                             article  \
0  By . Associated Press . PUBLISHED: . 14:11 EST...   
1  (CNN) -- Ralph Mata was an internal affairs li...   
2  A drunk driver who killed a young woman in a h...   
3  (CNN) -- With a breezy sweep of his pen Presid...   
4  Fleetwood are the only team still to have a 10...   

                                          highlights  
0  Bishop John Folda, of North Dakota, is taking ...  
1  Criminal complaint: Cop used his role to help ...  
2  Craig Eccleston-Todd, 27, had drunk at least t...  
3  Nina dos Santos says Europe must be ready to a...  
4  Fleetwood top of League One after 2-0 win at S.

## Задача 1: суммаризация

Эту задачу мы решим за вас. Последовательность следующая:

- создаем pipeline для суммаризации, указываем название модели
- берем первую новость из нашего датасета (текст этой новости)
- применяем модель суммаризации к этой новости, причем хотим саммари длины от 30 до 130 токенов. Смотрим результат.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
df['article'][0]

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [ ]:
summary = summarizer(df['article'][0], max_length=130, min_length=30, do_sample=False)
summary[0]['summary_text']

'Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.'

Задание: выведите в цикле для первых 10 новостей из датасета их текст и саммари.

In [ ]:
for i in range(10):

    article = df['article'][i]

    print(f"--- Обработка новости №{i+1} ---")

    if pd.isna(article) or not str(article).strip():
        print("! Статья пустая или отсутствует. Пропускаем.")
        print("-" * 80)
        continue

    try:
        # Генерация саммари
        summary = summarizer(
            article[:1024],
            max_length=130,
            min_length=30,
            do_sample=False
        )

        # Вывод результатов
        print("\nИсходный текст (первые 500 символов):")
        print(str(article)[:500])
        print("\nСаммари:")
        print(summary[0]['summary_text'])

    except Exception as e:
        print(f"! Произошла ошибка при обработке: {e}")

    print("-" * 80)

--- Обработка новости №1 ---

Исходный текст (первые 500 символов):
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in N

Саммари:
Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.
--------------------------------------------------------------------------------
--- Обработка новости №2 ---

Исходный текст (пе

## Задача 2: классификация

Теперь решим задачу классификации, а именно, анализа тональности новостей. Повторите последовательность действий:
- создайте pipeline для анализа тональности ("sentiment-analysis"), укажите название модели "cardiffnlp/twitter-roberta-base-sentiment-latest"
- возьмите первую новость из нашего датасета (текст этой новости)
- применяем модель классификации к этой новости, выведите результат


In [ ]:
# ваш код здесь
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
df['article'][0]

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [ ]:
sentiment = sentiment_pipeline(df['article'][0])
print(sentiment[0]['label'])
print(sentiment[0]['score'])

neutral
0.8927844166755676


Теперь в цикле примените классификатор к первым 10 новостям. Что произошло?

In [ ]:
# ваш код здесь
for i in range(10):
    # Получение статьи
    article = df['article'][i]

    # Генерация тональности
    sentiment = sentiment_pipeline(article)

    # Вывод результатов
    print(f"\nНовость №{i+1}:")
    print("\nИсходный текст:")
    print(article[:500])
    print("\nТональность:")
    print(sentiment[0]['label'])
    print("\nВероятность:")
    print(sentiment[0]['score'])
    print("-" * 80)


Новость №1:

Исходный текст:
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in N

Тональность:
neutral

Вероятность:
0.8927844166755676
--------------------------------------------------------------------------------

Новость №2:

Исходный текст:
(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan

RuntimeError: The expanded size of the tensor (1065) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1065].  Tensor sizes: [1, 514]

Для длинных новостей сначала сделайте суммаризацию, а затем применяйте классификатор. Проделайте это упражнение в цикле для первых 10 новостей. Теперь все получилось?

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

MAX_INPUT_CHARS = 4096

for i in range(10):
    article = df['article'][i]
    print(f"--- Обработка новости №{i+1} ---")

    try:

        article_text = str(article).strip()
        safe_article_chunk = article_text[:MAX_INPUT_CHARS]
        # Модель сама обрежет длинный текст, но лучше подавать ей разумный кусок
        summary_result = summarizer(
            safe_article_chunk,
            max_length=150,
            min_length=40,
            do_sample=False
        )
        summary_text = summary_result[0]['summary_text']


        sentiment = sentiment_pipeline(summary_text)


        print("\nИсходный текст (первые 500 символов):")
        print(f"{str(article)[:500]}...")

        print("\n Краткое содержание (Саммари):")
        print(summary_text)

        print("\n Тональность (по саммари):")
        label = sentiment[0]['label']
        score = sentiment[0]['score']


        print(f"Результат: {label}")
        print(f"Уверенность: {score:.2%}") # Выводим в процентах

    except Exception as e:
        print(f"! Произошла непредвиденная ошибка при обработке: {e}")

    finally:
        print("-" * 80 + "\n")

Device set to use cpu
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


--- Обработка новости №1 ---

Исходный текст (первые 500 символов):
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in N...

 Краткое содержание (Саммари):
Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.

 Тональность (по саммари):
Результат: neutral
Уверенность: 64.93%
-------------------------------------

## Задача 3: распознавание именованных сущностей (NER)

Наконец, решим задачу поиска именованных сущностей в текстах новостей (Named Entity Recognition, NER). Повторите последовательность действий:

- создайте pipeline для NER ("ner"), укажите название модели "dslim/bert-base-NER". Также в пайплайне укажите aggregation_strategy="simple" (это способ определения именованных сущностей для сущностей из нескольких слов)
- возьмите первую новость из нашего датасета (текст этой новости)
- примените модель к этой новости, выведите результат

In [ ]:
# ваш код здесь
ner_pipeline = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"
)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
df['article'][0]

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [ ]:
ner = ner_pipeline(df['article'][0])
for entity in ner:
        print(
            f"Сущность: {entity['word']}, "
            f"Тип: {entity['entity_group']}, "
            f"Уверенность: {entity['score']:.2%}"
        )

Сущность: Associated Press, Тип: ORG, Уверенность: 99.80%
Сущность: Fargo Catholic, Тип: ORG, Уверенность: 81.48%
Сущность: Diocese, Тип: LOC, Уверенность: 46.36%
Сущность: North Dakota, Тип: LOC, Уверенность: 99.94%
Сущность: Fargo, Тип: LOC, Уверенность: 99.76%
Сущность: Grand Forks, Тип: LOC, Уверенность: 99.57%
Сущность: Jamestown, Тип: LOC, Уверенность: 99.70%
Сущность: A, Тип: MISC, Уверенность: 49.28%
Сущность: Health Department, Тип: ORG, Уверенность: 99.66%
Сущность: John Folda, Тип: PER, Уверенность: 99.42%
Сущность: Fargo Catholic Diocese, Тип: ORG, Уверенность: 71.95%
Сущность: North Dakota, Тип: LOC, Уверенность: 99.94%
Сущность: Fargo, Тип: LOC, Уверенность: 99.82%
Сущность: Grand Forks, Тип: LOC, Уверенность: 99.79%
Сущность: Jamestown, Тип: LOC, Уверенность: 99.66%
Сущность: I, Тип: ORG, Уверенность: 70.50%
Сущность: Molly Howell, Тип: PER, Уверенность: 99.96%
Сущность: John Folda, Тип: PER, Уверенность: 89.86%
Сущность: Italy, Тип: LOC, Уверенность: 99.98%
Сущность: Fa

Примените модель к первым 10 новостям из датасета. Если возникнут проблемы - попробуйте их исправить.

In [ ]:
# ваш код здесь

for i in range(10):
    try:
        # Получение текста новости
        article = df['article'][i]

        # Проверка на пустые значения
        if pd.isna(article):
            print(f"\nНовость №{i+1}: отсутствует текст")
            continue

        # Обрезаем текст, если он слишком длинный
        safe_article = str(article)[:4096].strip()

        # Извлечение сущностей
        entities = ner_pipeline(safe_article)

        # Вывод результатов
        print(f"\n--- Новость №{i+1} ---")
        print(f"Первые 500 символов текста:\n{str(article)[:500]}...")
        print("\nНайденные сущности:")

        if not entities:
            print("Сущности не найдены")
            continue

        for entity in entities:
            print(
                f"  Сущность: {entity['word']}, "
                f"Тип: {entity['entity_group']}, "
                f"Уверенность: {entity['score']:.2%}"
            )

    except Exception as e:
        print(f"\nОшибка при обработке новости №{i+1}: {str(e)}")

    finally:
        print("-" * 80)


--- Новость №1 ---
Первые 500 символов текста:
By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in N...

Найденные сущности:
  Сущность: Associated Press, Тип: ORG, Уверенность: 99.80%
  Сущность: Fargo Catholic, Тип: ORG, Уверенность: 81.48%
  Сущность: Diocese, Тип: LOC, Уверенность: 46.36%
  Сущность: North Dakota, Тип: LOC, Уверенность: 99.94%
  Сущность: Fargo, Тип: LOC, Уверенность: 99.76%
  Сущность: Grand Forks, Тип: LOC, Уверенность: 99.57%
  Сущность: Jamestown, Тип: LOC, Уверенность: 99.70%
  Сущность: A, Тип: MISC, Уверенность: 49.28%

## Часть 2: применяем большие генеративные модели

Поработаем с моделью [OpenChat](https://huggingface.co/openchat/openchat-3.5-0106).

Загрузите модель (запустите ячейки ниже).

In [ ]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

71

In [ ]:
import transformers

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'openchat/openchat-3.5-0106'

tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

ModuleNotFoundError: Could not import module 'AutoModelForCausalLM'. Are this object's requirements defined correctly?

### Сколько гигабайт GPU заняла модель?

In [ ]:
# ваш код или проверка здесь

# Функция для подсчёта памяти
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def get_model_size_gb(model):
    total_params = count_parameters(model)
    total_bytes = total_params * 2
    return total_bytes / (1024 ** 3)  # Конвертация в гигабайты

# Подсчёт памяти
model_size_gb = get_model_size_gb(model)
print(f"Размер модели на GPU: {model_size_gb:.2f} ГБ")

# Дополнительно можно проверить использование памяти GPU
print("\nИспользование памяти GPU:")
print(f"Общая память GPU: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} ГБ")
print(f"Занято памяти GPU: {torch.cuda.memory_allocated() / (1024**3):.2f} ГБ")
print(f"Максимально использовалось памяти GPU: {torch.cuda.max_memory_allocated() / (1024**3):.2f} ГБ")

Размер модели на GPU: 13.49 ГБ

Использование памяти GPU:
Общая память GPU: 14.74 ГБ
Занято памяти GPU: 12.43 ГБ
Максимально использовалось памяти GPU: 12.43 ГБ


### Задание: с помощью OpenChat попробуйте решить все три задачи выше.

Решим первую задачу (анализ тональности) за вас. Мы попытались подобрать полезную инструкцию для модели.

Очень важно проверить, что среда выполнения в google colab - это GPU! Иначе генерация будет идти ооооочень медленно.

In [ ]:
prompt = "Classify the following article as positive, neutral or negative: " + df['article'][0]
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

Input batch (encoded): {'input_ids': tensor([[    1,  4950,  1575,   272,  2296,  5447,   390,  5278, 28725, 14214,
           442,  7087, 28747,  2463,   842,  6097,   601,  6127,   842,   367,
          6870, 28758,  1851, 22035, 28747,   842, 28705, 28740, 28781, 28747,
         28740, 28740,   413,   920, 28725, 28705, 28750, 28782,  4527, 28705,
         28750, 28734, 28740, 28770,   842,   342,   842,   500,  8268, 14371,
         28747,   842, 28705, 28740, 28782, 28747, 28770, 28784,   413,   920,
         28725, 28705, 28750, 28782,  4527, 28705, 28750, 28734, 28740, 28770,
           842,   415,   287, 12019,   302,   272,   401,  8473, 11377,  6216,
           402,  3368,   297,  3964, 27266,   659, 13438, 13957, 10524,   302,
          6059,  3338,   297,   401,  8473, 28725,  6607,   401,   580, 28713,
           304, 11776,   374,   656,   298,   272,   400,  3612, 17428,   330,
         15022,   297,  3909,  4074,   304,  2935,  4527, 28723,   415,  1665,
          6377,

In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=50, # максимальная длина ответа
    temperature=0.7, # можно менять этот параметр для настройки вариативности/точности результата
    top_p=0.9, # можно менять этот параметр для настройки вариативности/точности результата
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:] # выводим на экран только сгенерированный текст, без исходного
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("📄 Ответ модели:")
print(response.strip())

📄 Ответ модели:
The bishop's office confirmed that Bishop Folda has received the first dose of a hepatitis A vaccine . Bishop Foldo (sic) said in a statement: 'I am so sorry for the concern and distress this may


Получилось не очень :( Как будто модель не поняла, чего мы от нее хотим.

Мы можем дать модели примеры ответов в удобном для нее шаблоне. Попробуем!

In [ ]:
prompt = """
GPT4 Correct Assistant:
You are a helpful assistant that classifies the sentiment of news headlines as Positive, Neutral, or Negative.
Classify the sentiment of each news item as Positive, Neutral, or Negative.

GPT4 Correct User:
Question: Apple's new iPhone sales break records in first weekend.
GPT4 Correct Assistant:
Correct Answer: POSITIVE <|end_of_turn|>

GPT4 Correct User:
Question: The city council met Tuesday to discuss zoning regulations.
GPT4 Correct Assistant:
Correct Answer: NEUTRAL <|end_of_turn|>

GPT4 Correct User:
Question: Severe floods displace thousands in southern regions.
GPT4 Correct Assistant:
Correct Answer: NEGATIVE <|end_of_turn|>

GPT4 Correct User: Question: """ + df['article'][0] + """
GPT4 Correct Assistant:
""".strip()

batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

Input batch (encoded): {'input_ids': tensor([[    1, 28705,    13, 28777,  6316, 28781,  3198,  3123, 21631, 28747,
            13,  1976,   460,   264, 10865, 13892,   369,   875,  8961,   272,
         21790,   302,  4231,  1335,  8063,   390,  8202,  2468, 28725,  3147,
           329,  1650, 28725,   442, 12165,  1197, 28723,    13,  2472,  1575,
           272, 21790,   302,  1430,  4231,  2515,   390,  8202,  2468, 28725,
          3147,   329,  1650, 28725,   442, 12165,  1197, 28723,    13,    13,
         28777,  6316, 28781,  3198,  3123,  1247, 28747,    13, 24994, 28747,
         10244, 28742, 28713,   633, 17391,  6292,  1721,  7806,   297,   907,
          9071, 28723,    13, 28777,  6316, 28781,  3198,  3123, 21631, 28747,
            13,  9903,  3123, 26307, 28747,   367,  3843,  1153,  8138, 28705,
         32000, 28705,    13,    13, 28777,  6316, 28781,  3198,  3123,  1247,
         28747,    13, 24994, 28747,   415,  2990, 14725,  1424, 11187,   298,
          3342,

In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=50, # максимальная длина ответа
    temperature=0.7, # можно менять этот параметр для настройки вариативности/точности результата
    top_p=0.9, # можно менять этот параметр для настройки вариативности/точности результата
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:] # выводим на экран только сгенерированный текст, без исходного
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("📄 Ответ модели:")
print(response.strip())

📄 Ответ модели:
Correct Answer: NEGATIVE


Ваша очередь!

- Подберите промпт (текстовый запрос) для решения задачи суммаризации и запустите генерацию для первой новости из датасета
- Подберите промпт для решения задачи NER и запустите генерацию для первой новости из датасета

Оцените визуально результат и обязательно напишите текстом выводы:
- Хорошо ли справилась LLM?
- LLM справилась лучше, чем специализированные модели из части 1 или хуже?

In [ ]:
# ваш код здесь
# Промпт для суммаризации
summarization_prompt = f"""
GPT4 Summarizer:
You are an expert news summarizer. Create a concise summary of the news article in 3-4 sentences, preserving the main ideas and key facts.

Article: {df['article'][0]}

Summary:
"""

# Подготовка и генерация
batch = tokenizer(summarization_prompt, return_tensors='pt', return_token_type_ids=False).to(device)


In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=150,  # увеличено для более длинного ответа
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:]
summary = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("\n📊 Сгенерированная суммаризация:")
print(summary.strip())


📊 Сгенерированная суммаризация:
Bishop John Folda of the Fargo Catholic Diocese in North Dakota has potentially exposed hundreds of church members in Fargo, Grand Forks, and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for those who attended five churches and took communion. The risk is considered low, but officials feel it's important to alert people to the possible exposure. Folda contracted the infection through contaminated food while attending a conference in Italy last month.

Note: This summary is not a direct transcription of the article, but a concise summary of the main ideas and key facts.

---

G


In [ ]:
# Промпт для NER
ner_prompt = f"""
Named Entity Recognizer:
Extract all named entities from the text and classify them into categories:
- PERSON (люди)
- ORG (организации)
- LOC (места)
- DATE (даты)
- MONEY (денежные суммы)
- EVENT (события)

Format your response as a list of entities with their types:

Text: {df['article'][0]}

Entities:
"""

# Подготовка и генерация
batch = tokenizer(ner_prompt, return_tensors='pt', return_token_type_ids=False).to(device)

In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=200,  # увеличено для более подробного ответа
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:]
entities = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("\n🔍 Извлеченные сущности:")
print(entities.strip())


🔍 Извлеченные сущности:
- Associated Press (ORG)
- North Dakota (LOC)
- Fargo Catholic Diocese (ORG)
- Fargo (LOC)
- Grand Forks (LOC)
- Jamestown (LOC)
- October 25 (DATE)
- Bishop John Folda (PERSON)
- Molly Howell (PERSON)
- State Immunization Program (ORG)
- Italy (LOC)
- hepatitis A (EVENT)

The code to run the Named Entity Recognizer is below:

```python
import spacy

nlp = spacy.load("en_core_web_sm")

text = """By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 E


# Выводы

LLM справилась с задачей суммаризации, но в отличии от специализированных моделей, добавила всё-таки лишнюю информацию. Она хуже концентрируется на основных фактах, добавляя лишние детали

Базовая суммаризация за счет моделей справилась всё-таки лучше, чем LLM.

С задачей NER LLM также справилась хуже моделей. Во-первых, она нашла меньше корректных именованных сущностей. Ошибки в определении нашлись в обоих случаях. Можно сказать, что LLM нашла те сущности, в которых у специализированных моделей уверенность около 99%. А сущности с уверенностью, например, 90% LLM найти уже не смогла

## Часть 3: творческая

Попробуйте [собрать RAG-систему](https://colab.research.google.com/drive/196TKVfLWesbrF7f4KNHMGZNJC3hfg5IK?usp=sharing), как мы делали это на вебинаре, для модели OpenChat (можно использовать код по ссылке, только внимательно смотрите, все ли параметры актуальны именно для этой модели).

В качестве данных давайте возьмем [rag-mini-bioasq](https://huggingface.co/datasets/enelpol/rag-mini-bioasq), это структурно схожый датасет с тем, что был в семинаре, но теперь он по био-медицинской тематике. Это сабдатасет с вокршопа [the BioASQ Challenge](https://www.bioasq.org/), то есть вы решаете задачу самого актуального соревнования в сфере био-медицинского NLP!

Основная сложность, с которой вы сталкнетесь, как оценить получившуюся RAG-систему. Авторы соревнования делают это [так](http://participants-area.bioasq.org/Tasks/b/eval_meas_2022/): они разбивают валидацию на две части - оценка retrievel и оценка сгенерированного ответа. Давайте сделаем так же, но упростим нашу систему.

Оба способа оценки вы реализуете самостоятельно. Для части с answer давайте отфильтруем вопросы формата yes/no и просто посчитаем F1 по двум классам. А для части с retrievel используем метрику Hit Rate@k - попал ли хоть один релевантный документ в первые *k*? Форма будет следующая для одного вопроса *i*:

Hit Rate_i\@k

  $$
    \text{success}_i =
      \begin{cases}
        1, & \text{если } |\,\text{gold}_i \cap \text{retrieved}_i^{[:k]}| > 0 \\
        0, & \text{иначе}
      \end{cases}
  $$

Затем агрегируем по всем *N* вопросам:

$$
  \text{Hit Rate@k} = \frac{1}{N}\sum_{i=1}^{N}\text{success}_i.
$$

Другими словами, мы просто для каждого вопроса считаем, попал ли он в топ_k или нет, а потом усредняем по всем вопросам. Давайте в качестве k возьмем 3, то есть будем считать Hit Rate@3.

*Пример*

| Вопрос | Релевантных доков (gold) | Что вернула система (топ-3) | Hit Rate\@3   
| ------ | ------------------------ | --------------------------- | ------------
| Q1     | {A, B}                   | A, C, D                     | 1 (A найден)
| Q2     | {E}                      | F, G, H                     | 0            
| Q3     | {J, K, L}                | K, L, A                     | 1 (A найден)      

Средние значения: *Hit Rate\@3* = (1 + 0 + 1)/3 = 0.67

Осталось понять, где взять золотой ответ и что делать, если вы используете разбиение на чанки. 1) в датасете есть relevant_passage_ids - это и есть золотые (golden) ответы. 2) при разбиении на чанки сохраняйте информацию о том, к какого relevant_passage_ids он относится. Например, вот так: metadata={"id": rec["id"], "chunk": i}.

Если вы вдруг в процессе реализации задания столкнетесь со сложностями, не бойтесь подглядывать в решения других датасаентистов. Например, [kaggle](https://www.kaggle.com/) - это кладезь отличных готовых решений различных задач, и RAG в том числе. Можно, допустим, заглянуть в [ноутбук](https://www.kaggle.com/code/erwanversmee/rag-for-mini-bioasq/notebook) к Erwan Versmee (если найдете решение полезным, поставьте лайк - человеку будет приятно). Каждый код индивидуален, и чем больше вы увидите разных вариаций, тем сильнее станете как специалист. Удачи в решении задания!

# Pips

In [ ]:
# ваш код здесь

!pip install -q --upgrade pip
!pip uninstall -y fastai torch torchvision torchaudio gcsfs fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.9 MB/s eta 0:00:00
Found existing installation: fastai 2.7.19
Uninstalling fastai-2.7.19:
  Successfully uninstalled fastai-2.7.19
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: gcsfs 2025.3.2
Uninstalling gcsfs-2025.3.2:
  Successfully uninstalled gcsfs-2025.3.2
Found existing installation: fsspec 2025.3.2
Uninstalling fsspec-2025.3.2:
  Successfully uninstalled fsspec-2025.3.2


In [ ]:
!pip install --upgrade transformers torch bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 \
    --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'lit' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'lit'. Discussion can be found at https://github.com/pypa/pip/issues/6334
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [torchaudio]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [ ]:
!pip install -q fsspec==2025.3.2 gcsfs==2025.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gcsfs]


In [ ]:
!pip install -q \
    bitsandbytes==0.46.0 \
    transformers==4.52.4 \
    accelerate==1.8.1 \
    datasets==3.6.0 \
    sentence-transformers==4.1.0 \
    chromadb==1.0.13 \
    langchain==0.3.26 \
    langchain-community==0.3.26 \
    langchain-huggingface \
    tqdm==4.67.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38/38 [langchain-community]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 2.7.1

In [ ]:
!pip uninstall transformers bitsandbytes
!pip install transformers==4.32.0 bitsandbytes
!pip install langchain_community chromadb sentence-transformers

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Would remove:
    /usr/local/bin/transformers
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.52.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.52.4
Found existing installation: bitsandbytes 0.46.0
Uninstalling bitsandbytes-0.46.0:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/bitsandbytes-0.46.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/bitsandbytes/*
Proceed (Y/n)? y
  Successfully uninstalled bitsandbytes-0.46.0
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.2 MB/s eta 0:00:00
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)
  Attempting unin

# BitsAndBytes

In [ ]:
from torch import cuda, bfloat16
import torch, transformers
import torchvision
from torch.optim.lr_scheduler import LRScheduler

/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
# from torch import cuda, bfloat16
# import torch, transformers
# import torchvision
# from torch.optim.lr_scheduler import LRScheduler
from transformers import AutoTokenizer, BitsAndBytesConfig, pipeline


torchvision.disable_beta_transforms_warning()
DEVICE = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"

/usr/local/lib/python3.11/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [ ]:
import sys
print(transformers.__version__, sys.executable)

4.53.3 /usr/bin/python3


In [ ]:
import bitsandbytes as bnb, importlib, os
print(bnb.__version__)
!ldconfig -p | grep cusparse | head -n 3

0.46.1
	libcusparse.so.12 (libc6,x86-64) => /usr/local/cuda/targets/x86_64-linux/lib/libcusparse.so.12
	libcusparse.so (libc6,x86-64) => /usr/local/cuda/targets/x86_64-linux/lib/libcusparse.so


# LLM

In [ ]:
model_name = 'openchat/openchat-3.5-0106'

# Квантуем в 4 бита, чтобы поместилось в VRAM 6–8 ГБ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                          # включить 4-битное квантование
    bnb_4bit_quant_type="nf4",                  # тип квантования: "nf4" (Normalized Float 4) или "fp4"
    bnb_4bit_use_double_quant=True,             # включить двойное квантование (дополнительная компрессия)
    bnb_4bit_compute_dtype=torch.float16        # тип данных для вычислений (например, bfloat16 (недоступен на T4), float16)
)


print("Загружаем модель …")
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    # trust_remote_code=True,
)

# tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name)
# tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id


Загружаем модель …


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

# Данные

In [ ]:
from pathlib import Path

from datasets import load_dataset
from langchain.docstore.document import Document

from langchain.document_loaders import TextLoader  # загружает текстовые файлы и превращает их в объекты Document для LangChain.
from langchain.text_splitter import RecursiveCharacterTextSplitter  # рекурсивно разбивает длинный текст на более мелкие фрагменты (chunks).
from langchain_huggingface import HuggingFaceEmbeddings  # оборачивает модели из HuggingFace для получения эмбеддингов текста.
from langchain.vectorstores import Chroma  # векторное хранилище Chroma: сохраняет и ищет эмбеддинги.

from langchain_huggingface import HuggingFacePipeline  # использует HuggingFace Transformers pipeline как LLM-модуль в LangChain.
from langchain.chains import RetrievalQA  # готовая цепочка «поиск + генерация ответа» (Retrieval-augmented QA).

In [ ]:
bio_ds = load_dataset("rag-datasets/rag-mini-bioasq", name="text-corpus", split="passages")

# Создание документов
corpus_docs = [
    Document(page_content=rec["passage"], metadata={"id": rec["id"]})
    for rec in bio_ds
]

print(f"Загружено документов: {len(corpus_docs)}")
print("Пример документа:\n", corpus_docs[0].page_content[:200], "...")

README.md:   0%|          | 0.00/813 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/24.5M [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/40221 [00:00<?, ? examples/s]

Загружено документов: 40221
Пример документа:
 New data on viruses isolated from patients with subacute thyroiditis de Quervain 
are reported. Characteristic morphological, cytological, some physico-chemical 
and biological features of the isolate ...


In [ ]:
bio_ds

Dataset({
    features: ['passage', 'id'],
    num_rows: 40221
})

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=50
)

# docs = splitter.split_documents(corpus_docs)
# print("Чанков:", len(docs))

docs = []
for doc in corpus_docs:
    chunks = splitter.split_documents([doc])
    for i, chunk in enumerate(chunks):
        chunk.metadata["chunk"] = i
        docs.append(chunk)

print("Чанков:", len(docs))

Чанков: 111982


# Создание векторной базы

In [ ]:
persist_directory = "chroma_ragmini"
if os.path.exists(persist_directory):
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
else:
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2",
        model_kwargs={"device": "cuda"}
    )
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vectordb.persist()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
/tmp/ipython-input-15-2356646682.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


# Сборка цепочки

In [ ]:
import numpy as np

In [ ]:
import ast

def evaluate_retrieval(retriever, val_ds, k=3):
    # Загружаем только первые 10 примеров
    val_ds = load_dataset("rag-datasets/rag-mini-bioasq",
                         name="question-answer-passages",
                         split="test").select(range(10))

    hit_rates = []
    results = []  # Для хранения детальной информации

    for example in val_ds:
        question = example["question"]

        try:
            # Преобразуем строку в список чисел
            relevant_ids_str = example["relevant_passage_ids"]
            gold_ids = set(ast.literal_eval(relevant_ids_str))

            # Получаем топ-k документов
            docs = retriever.get_relevant_documents(question)[:k]
            retrieved_ids = set(doc.metadata["id"] for doc in docs)

            # Проверяем пересечение с золотыми id
            hit = 1 if gold_ids.intersection(retrieved_ids) else 0
            hit_rates.append(hit)

            # Сохраняем детальную информацию
            results.append({
                "question": question,
                "gold_ids": gold_ids,
                "retrieved_ids": retrieved_ids,
                "hit": hit
            })

        except Exception as e:
            print(f"Ошибка при обработке вопроса '{question}': {e}")
            hit_rates.append(0)
            results.append({
                "question": question,
                "error": str(e)
            })

    # Выводим детальную информацию
    print("\nПодробные результаты:")
    for res in results:
        print(f"\nВопрос: {res['question']}")
        print(f"Золотые ID: {res['gold_ids']}")
        print(f"Найденные ID: {res['retrieved_ids']}")
        print(f"Hit: {'Да' if res.get('hit', 0) else 'Нет'}")

    return np.mean(hit_rates) if hit_rates else 0.0

In [ ]:

def evaluate_answers(qa_chain, val_ds):
    val_ds = load_dataset("rag-datasets/rag-mini-bioasq",
                         name="question-answer-passages",
                         split="test").select(range(10))

    true_labels = []
    predicted_labels = []
    results = []

    print("\nРезультаты оценки:")

    for example in val_ds:
        question = example["question"]
        gold_answer = example["answer"].strip().lower()

        # Определяем тип вопроса
        is_yes_no = gold_answer in ["yes", "no"]

        try:
            response = qa_chain.invoke({"query": question})["result"].strip().lower()

            if is_yes_no:
                true_label = 1 if gold_answer == "yes" else 0
                predicted = 1 if response == "yes" else 0

                true_labels.append(true_label)
                predicted_labels.append(predicted)

                results.append({
                    "question": question,
                    "gold_answer": gold_answer,
                    "model_response": response,
                    "correct": true_label == predicted
                })

                print(f"\nВопрос: {question}")
                print(f"Золотой ответ: {gold_answer}")
                print(f"Ответ модели: {response}")
                print(f"Верно: {'Да' if true_label == predicted else 'Нет'}")

            else:
                # print(f"\nВопрос: {question}")
                # print(f"Золотой ответ: {gold_answer}")
                # print(f"Ответ модели: {response}")
                print("Это не yes/no вопрос, оценка не производится")

        except Exception as e:
            print(f"Ошибка при обработке вопроса '{question}': {e}")

    if true_labels:
        f1 = f1_score(true_labels, predicted_labels)
        print(f"\nF1 score для yes/no вопросов: {f1:.4f}")
    else:
        print("\nНет yes/no вопросов в выборке")

    return f1

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from transformers import GenerationConfig
from langchain import PromptTemplate


In [ ]:
gen_cfg = GenerationConfig(
    max_length=2048,
    max_new_tokens=128,
    # temperature=0.1,
    # do_sample=False,
    # top_p=0.9
)

In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto"
    ),
    model_kwargs={
        "max_new_tokens": 128,
        "temperature": 0.1,
        "do_sample": False
    }
)

Device set to use cuda:0


In [ ]:
print(val_ds.column_names)

['passage', 'id']


In [ ]:
# Обновленный промпт
# template = """
# Answer the question based on the provided context. Follow these rules strictly:

# 1. For yes/no questions:
#    - Respond ONLY with "Yes" or "No"
#    - No additional explanations

# 2. For open-ended questions:
#    - Provide a concise answer (1-2 sentences)
#    - Use only information from the context
#    - Keep the answer relevant and to the point

# 3. DO NOT include:
#    - Any instructions
#    - Role descriptions
#    - Extra text

# Context:
# {context}

# Question: {question}

# Answer:
# """
# prompt = PromptTemplate(
#     template=template,
#     input_variables=["context", "question"]
# )


In [ ]:
# Максимально упрощенный промпт
template = """
Answer ONLY the question with a direct response.

Question: {question}
Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

In [ ]:
val_ds = load_dataset("rag-datasets/rag-mini-bioasq", name="question-answer-passages", split="test").select(range(10))

# test_samples = val_ds.select(range(10))

part.0.parquet:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4719 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(val_ds):
        if i >= 10:  # Ограничение до 10 примеров
            break

        question = example["question"]
        answer = example["answer"].lower()

        print(f"\nВопрос {i+1}: {question}")
        print(f"Ответ: {answer}")

        # Проверяем, является ли вопрос yes/no
        if answer in ["yes", "no"]:
            print("Тип вопроса: Yes/No")
        else:
            print("Тип вопроса: Открытый")


Вопрос 1: Is Hirschsprung disease a mendelian or a multifactorial disorder?
Ответ: coding sequence mutations in ret, gdnf, ednrb, edn3, and sox10 are involved in the development of hirschsprung disease. the majority of these genes was shown to be related to mendelian syndromic forms of hirschsprung's disease, whereas the non-mendelian inheritance of sporadic non-syndromic hirschsprung disease proved to be complex; involvement of multiple loci was demonstrated in a multiplicative model.
Тип вопроса: Открытый

Вопрос 2: List signaling molecules (ligands) that interact with the receptor EGFR?
Ответ: the 7 known egfr ligands  are: epidermal growth factor (egf), betacellulin (btc), epiregulin (epr), heparin-binding egf (hb-egf), transforming growth factor-α [tgf-α], amphiregulin (areg) and epigen (epg).
Тип вопроса: Открытый

Вопрос 3: Is the protein Papilin secreted?
Ответ: yes,  papilin is a secreted protein
Тип вопроса: Открытый

Вопрос 4: Are long non coding RNAs spliced?
Ответ: long n

In [ ]:
from langchain.chains import LLMChain

# Создаем цепочку
chain = LLMChain(llm=llm, prompt=prompt)

def evaluate_single_answer(chain, question, gold_answer):
    try:
        # Получаем полный ответ модели
        full_response = chain.run(question)

        print(f"\nПОЛНЫЙ ОТВЕТ МОДЕЛИ: {full_response}")

        # Очищаем ответ от лишних символов
        cleaned_response = full_response.strip().lower().split()[0] if full_response else ""

        # Определяем тип вопроса
        is_yes_no = gold_answer.lower() in ["yes", "no"]

        if is_yes_no:
            true_label = 1 if gold_answer.lower() == "yes" else 0
            predicted = 1 if cleaned_response == "yes" else 0

            print(f"\nВопрос: {question}")
            print(f"Золотой ответ: {gold_answer}")
            print(f"Очищенный ответ модели: {cleaned_response}")
            print(f"Верно: {'Да' if true_label == predicted else 'Нет'}")

            return true_label, predicted
        else:
            print("Это не yes/no вопрос")
            return None, None

    except Exception as e:
        print(f"Ошибка при обработке вопроса: {e}")
        return None, None

# Тестируем на конкретном примере
question = "Is the protein Papilin secreted?"
gold_answer = "Yes"

true_label, predicted = evaluate_single_answer(chain, question, gold_answer)

KeyboardInterrupt: 

In [ ]:
hit_rate = evaluate_retrieval(retriever, val_ds, k=3)
print(f"Hit Rate@3: {hit_rate:.4f}")

answer_f1 = evaluate_answers(qa_chain, val_ds)
print(f"F1 score для yes/no вопросов: {answer_f1:.4f}")


Подробные результаты:

Вопрос: Is Hirschsprung disease a mendelian or a multifactorial disorder?
Золотые ID: {20598273, 6650562, 15829955, 15617541, 23001136, 8896569, 21995290, 12239580, 15858239}
Найденные ID: {23001136, 10964697, 17108762}
Hit: Да

Вопрос: List signaling molecules (ligands) that interact with the receptor EGFR?
Золотые ID: {23821377, 24323361, 22247333, 23787814, 22260327, 23888072, 23959273, 24124521, 23729230, 23089711, 21514161, 24204699, 23212918, 23099994, 23382875, 23399900}
Найденные ID: {16159418, 23382875}
Hit: Да

Вопрос: Is the protein Papilin secreted?
Золотые ID: {21784067, 19297413, 15094122, 3320045, 7515725, 20805556, 19724244, 12666201, 15094110, 11076767}
Найденные ID: {15094122, 3320045, 11076767}
Hit: Да

Вопрос: Are long non coding RNAs spliced?
Золотые ID: {22955974, 21622663, 22707570, 22955988, 24285305, 24106460}
Найденные ID: {24285305, 25483404, 22955974}
Hit: Да

Вопрос: Is RANKL secreted from the cells?
Золотые ID: {22867712, 23827649, 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Результаты оценки:
Это не yes/no вопрос, оценка не производится
Это не yes/no вопрос, оценка не производится
Это не yes/no вопрос, оценка не производится
Это не yes/no вопрос, оценка не производится
Это не yes/no вопрос, оценка не производится


KeyboardInterrupt: 

# Результаты

Мне удалось достигнуть метрики hit rate@k = 0.7

ID документов действительно верно находятся во всех правильных ответах.

Но, к сожалению, мне не удалось подстроить промпт и функцию для оценку yes/no ответов так, чтобы модель считывала их как подобные вопросы. Она всегда добавляла к ответам лишний контекст, хоть и сами ответы  yes/no в дальнейшем совпадали

Поэтому оценить f1-score можно было только вручную, что я, к сожалению, тоже забыл сделать перед изменениями (а времени на gpu для перезапуска оставалось немного). Но по моей памяти все ответы, которые я видел(5-10 штук), совпадали.